<a href="https://colab.research.google.com/github/ArpanSM/MachineLearningHackathons/blob/master/Used_Electronics_Price_Prediction_Weekend_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Data

In [1]:
!wget https://www.machinehack.com/wp-content/uploads/2020/06/Used_Electronics_Participants_Data.zip

--2020-06-07 19:16:54--  https://www.machinehack.com/wp-content/uploads/2020/06/Used_Electronics_Participants_Data.zip
Resolving www.machinehack.com (www.machinehack.com)... 107.180.71.20
Connecting to www.machinehack.com (www.machinehack.com)|107.180.71.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184210 (180K) [application/zip]
Saving to: ‘Used_Electronics_Participants_Data.zip’

Used_Electronics_Pa 100%[===================>] 179.89K   378KB/s    in 0.5s    

2020-06-07 19:16:54 (378 KB/s) - ‘Used_Electronics_Participants_Data.zip’ saved [184210/184210]



In [2]:
!unzip Used_Electronics_Participants_Data.zip

Archive:  Used_Electronics_Participants_Data.zip
   creating: Used_Electronics_Participants_Data/
  inflating: Used_Electronics_Participants_Data/Test.csv  
  inflating: __MACOSX/Used_Electronics_Participants_Data/._Test.csv  
  inflating: Used_Electronics_Participants_Data/Sample_Submission.xlsx  
  inflating: __MACOSX/Used_Electronics_Participants_Data/._Sample_Submission.xlsx  
  inflating: Used_Electronics_Participants_Data/Train.csv  
  inflating: __MACOSX/Used_Electronics_Participants_Data/._Train.csv  


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


In [0]:
df = pd.read_csv("/content/Used_Electronics_Participants_Data/Train.csv")
dft = pd.read_csv("/content/Used_Electronics_Participants_Data/Test.csv")

In [6]:
df.head()

,Brand,Model_Info,Additional_Description,Locality,City,State,Price
0,1,name0 name234 64gb space grey,1yesr old mobile number 999two905two99 bill c...,878,8,2,15000
1,1,phone 7 name42 name453 new condition box acce...,101004800 1010065900 7000,1081,4,0,18800
2,1,name0 x 256gb leess used good condition,1010010000 seperate screen guard 3 back cover...,495,11,4,50000
3,1,name0 6s plus 64 gb space grey,without 1010020100 id 1010010300 colour 10100...,287,10,7,16500
4,1,phone 7 sealed pack brand new factory outet p...,101008700 10100000 xs max 64 gb made 10100850...,342,4,0,26499


In [0]:
S = ['16gb','16 gb','32gb','32 gb','64gb','64 gb','128gb','128 gb','256gb','256 gb','512gb','512 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in S)
df['Storage'] = df['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
df['Storage1'] = df['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

df.Storage.fillna(df.Storage1, inplace=True)
del df['Storage1']

df["Storage"].replace({"16gb": 16,
                           "16 gb": 16,
                           "32gb": 32,
                           "32 gb": 32,
                           "64gb": 64,
                           "64 gb": 64,
                           "128 gb": 128,
                           "128gb": 128,
                           "256 gb": 256,
                           "256gb": 256,
                           "512 gb": 512,
                           "512gb": 512}, inplace=True)  


S = ['16gb','16 gb','32gb','32 gb','64gb','64 gb','128gb','128 gb','256gb','256 gb','512gb','512 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in S)
dft['Storage'] = dft['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
dft['Storage1'] = dft['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

dft.Storage.fillna(dft.Storage1, inplace=True)
del dft['Storage1']

dft["Storage"].replace({"16gb": 16,
                           "16 gb": 16,
                           "32gb": 32,
                           "32 gb": 32,
                           "64gb": 64,
                           "64 gb": 64,
                           "128 gb": 128,
                           "128gb": 128,
                           "256 gb": 256,
                           "256gb": 256,
                           "512 gb": 512,
                           "512gb": 512}, inplace=True)  

In [8]:
df.head(5)

,Brand,Model_Info,Additional_Description,Locality,City,State,Price,Storage
0,1,name0 name234 64gb space grey,1yesr old mobile number 999two905two99 bill c...,878,8,2,15000,64.0
1,1,phone 7 name42 name453 new condition box acce...,101004800 1010065900 7000,1081,4,0,18800,NaN
2,1,name0 x 256gb leess used good condition,1010010000 seperate screen guard 3 back cover...,495,11,4,50000,256.0
3,1,name0 6s plus 64 gb space grey,without 1010020100 id 1010010300 colour 10100...,287,10,7,16500,64.0
4,1,phone 7 sealed pack brand new factory outet p...,101008700 10100000 xs max 64 gb made 10100850...,342,4,0,26499,64.0


In [0]:
!pip install h2o
import h2o
h2o.init()
from h2o.automl import H2OAutoML

In [0]:
# applying groupby() function to group the data on team value. 
gk = df.groupby('Brand') 
model0 = gk.get_group(0)
model1 = gk.get_group(1) 
model2 = gk.get_group(2)
model3 = gk.get_group(3) 

gk_test = dft.groupby('Brand') 
model0_test = gk_test.get_group(0)
model1_test = gk_test.get_group(1) 
model2_test = gk_test.get_group(2)
model3_test = gk_test.get_group(3) 

##Model0

### Data PreProcessing

In [0]:
indices= model0['Price'] <= 5500
model0.loc[indices,'Storage'] = model0.loc[indices,'Storage'].fillna(16)

indices=  model0['Price'] <= 11500
model0.loc[indices,'Storage'] = model0.loc[indices,'Storage'].fillna(32)

model0['Storage'].fillna(64 , inplace=True)

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model0['RAM1'] = model0['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model0['RAM2'] = model0['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

model0.RAM1.fillna(model0.RAM2, inplace=True)
del model0['RAM2']


model0["RAM1"].replace({"2gb": 2,
                           "2 gb": 2,
                           "3gb": 3,
                           "3 gb": 3,
                           "4gb": 4,
                           "4 gb": 4,
                           "6 gb": 6,
                           "6gb": 6}, inplace=True)  

indices= model0['Storage'] == 16
model0.loc[indices,'RAM1'] = model0.loc[indices,'RAM1'].fillna(2)

indices= model0['Storage'] == 32
model0.loc[indices,'RAM1'] = model0.loc[indices,'RAM1'].fillna(3)

indices= model0['Storage'] == 64
model0.loc[indices,'RAM1'] = model0.loc[indices,'RAM1'].fillna(4)

model0['RAM1'].fillna(6 , inplace=True)

M = ['10 lite','10lite','p20 lite','9 lite','p20lite','9lite','8 lite','8lite','7x','7c','7a', '8c,''8x','nova','view','mate','9','9n','6x','8','7','10','5','5x','5 x','honor']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model0['Model1'] = model0['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model0['Model2'] = model0['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

model0.Model1.fillna(model0.Model2, inplace=True)
del model0['Model2']

model0["Model1"].replace({"10 lite": "10L", "10lite": "10L", "10": "10L",
                        "9 lite": "9L", "9lite": "9L", "9": "9L","9n": "9L",
                        "8 lite": "8L", "8lite": "8L", "8": "8L","8x": "8L","8c": "8L",
                        "7x": "7L", "7c": "7L", "7": "7L", "7a": "7L",
                        "6x": "6L",
                        "p20 lite": "P2L","p20lite": "P2L",
                        "5x": "5L", "5 x": "5L", "5": "5L"
                        }, inplace=True)

model0['Model1'].fillna("Other",inplace=True)   


M = ['10 lite','10lite','p20 lite','9 lite','p20lite','9lite','8 lite','8lite','7x','7c','7a', '8c,''8x','nova','view','mate','9','9n','6x','8','7','10','5','5x','5 x','honor']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model0_test['Model1'] = model0_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model0_test['Model2'] = model0_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

model0_test.Model1.fillna(model0_test.Model2, inplace=True)
del model0_test['Model2']

model0_test["Model1"].replace({"10 lite": "10L", "10lite": "10L", "10": "10L",
                        "9 lite": "9L", "9lite": "9L", "9": "9L","9n": "9L",
                        "8 lite": "8L", "8lite": "8L", "8": "8L","8x": "8L","8c": "8L",
                        "7x": "7L", "7c": "7L", "7": "7L", "7a": "7L",
                        "6x": "6L",
                        "p20 lite": "P2L","p20lite": "P2L",
                        "5x": "5L", "5 x": "5L", "5": "5L"
                        }, inplace=True)

model0_test['Model1'].fillna("Other",inplace=True)   

indices= model0_test['Model1'] == "10L"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(32)

indices= model0_test['Model1'] == "9L"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(64)

indices= model0_test['Model1'] == "7L"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(64)

indices= model0_test['Model1'] == "6L"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(64)

indices= model0_test['Model1'] == "mate"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(128)

indices= model0_test['Model1'] == "honor"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(32)

indices= model0_test['Model1'] == "Other"
model0_test.loc[indices,'Storage'] = model0_test.loc[indices,'Storage'].fillna(32)

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model0_test['RAM1'] = model0_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model0_test['RAM2'] = model0_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)

model0_test.RAM1.fillna(model0_test.RAM2, inplace=True)
del model0_test['RAM2']


model0_test["RAM1"].replace({"2gb": 2,
                           "2 gb": 2,
                           "3gb": 3,
                           "3 gb": 3,
                           "4gb": 4,
                           "4 gb": 4,
                           "6 gb": 6,
                           "6gb": 6}, inplace=True)  

indices= model0_test['Storage'] == 16
model0_test.loc[indices,'RAM1'] = model0_test.loc[indices,'RAM1'].fillna(2)

indices= model0_test['Storage'] == 32
model0_test.loc[indices,'RAM1'] = model0_test.loc[indices,'RAM1'].fillna(3)

indices= model0_test['Storage'] == 64
model0_test.loc[indices,'RAM1'] = model0_test.loc[indices,'RAM1'].fillna(4)

model0_test['RAM1'].fillna(6 , inplace=True)


In [0]:
del model0['Additional_Description']
del model0['Model_Info']
del model0_test['Additional_Description']
del model0_test['Model_Info']

In [0]:
model0.to_csv("model0.csv",index=True)
model0_test.to_csv("model0_test.csv",index=True)

### Modelling

In [138]:
df_model0 = h2o.import_file('/content/model0.csv')
dft_model0 = h2o.import_file('/content/model0_test.csv')

df_model0['Brand'] = df_model0['Brand'].asfactor()
df_model0['Locality'] = df_model0['Locality'].asfactor()
df_model0['State'] = df_model0['State'].asfactor()
dft_model0['Brand'] = dft_model0['Brand'].asfactor()
dft_model0['Locality'] = dft_model0['Locality'].asfactor()
dft_model0['State'] = dft_model0['State'].asfactor()
ignored_col_list = "C1"

print(df_model0.types)
print(dft_model0.types)

df_train, df_val, df_test = df_model0.split_frame(ratios=[.9,.05])
y = "Price"
x = df_model0.columns
x.remove(y)
x.remove(ignored_col_list)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Price': 'int', 'Storage': 'int', 'RAM1': 'int', 'Model1': 'enum'}
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}


In [139]:
aml = H2OAutoML(nfolds=15,
                balance_classes=False,
                max_runtime_secs=240,
                max_models=0,
                exclude_algos=['DeepLearning','GLM'],
                verbosity = "info",
                sort_metric='RMSLE',
                stopping_metric='RMSLE')
aml.train(x=x,y=y,training_frame=df_train,validation_frame=df_val)
aml.leaderboard

AutoML progress: |
20:56:57.708: Project: AutoML_20200607_205657708
20:56:57.708: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:56:57.708: Setting stopping tolerance adaptively based on the training frame: 0.05
20:56:57.708: Build control seed: -1 (random)
20:56:57.708: training frame: Frame key: automl_training_py_4566_sid_ab9d    cols: 9    rows: 74  chunks: 1    size: 9482  checksum: -1660705785103500
20:56:57.708: validation frame: Frame key: py_4567_sid_ab9d    cols: 9    rows: 7  chunks: 1    size: 8680  checksum: -343692596488320
20:56:57.708: leaderboard frame: NULL
20:56:57.708: blending frame: NULL
20:56:57.708: response column: Price
20:56:57.708: fold column: null
20:56:57.708: weights column: null
20:56:57.708: Loading execution steps: [{XGBoost : defaults}, {GL

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
XGBoost_grid__1_AutoML_20200607_205657_model_19,0.470372,1.02769e+08,10137.5,1.02769e+08,4416.58
XGBoost_grid__1_AutoML_20200607_205657_model_16,0.482944,9.6073e+07,9801.68,9.6073e+07,4491.82
XGBoost_grid__1_AutoML_20200607_205657_model_26,0.486055,1.01381e+08,10068.8,1.01381e+08,4556.9
XGBoost_grid__1_AutoML_20200607_205657_model_25,0.49233,1.06991e+08,10343.6,1.06991e+08,4669.7
GBM_grid__1_AutoML_20200607_205657_model_5,0.493397,1.00467e+08,10023.3,1.00467e+08,4533.88
DRF_1_AutoML_20200607_205657,0.493407,1.06581e+08,10323.8,1.06581e+08,4367.9
XGBoost_grid__1_AutoML_20200607_205657_model_3,0.496556,9.72985e+07,9864,9.72985e+07,4870.54
XGBoost_grid__1_AutoML_20200607_205657_model_36,0.496837,9.68252e+07,9839.98,9.68252e+07,4910.79
GBM_grid__1_AutoML_20200607_205657_model_18,0.520412,1.00892e+08,10044.5,1.00892e+08,5009.45
XRT_1_AutoML_20200607_205657,0.520709,1.16616e+08,10798.9,1.16616e+08,5374.38


In [140]:
pred = aml.leader.predict(df_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [141]:
pred.head()

predict
9545.83
12955.6
5635.13
12776.3


In [142]:
aml.leader.model_performance(df_test)


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 14036578.628154576
RMSE: 3746.5422229243027
MAE: 2500.3978271484375
RMSLE: 0.4330040620730148
Mean Residual Deviance: 14036578.628154576


In [143]:
dft_model0['predict'] = aml.leader.predict(dft_model0)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [144]:
dft_model0

C1,Brand,Locality,City,State,Storage,Model1,RAM1,predict
15,0,361,0,1,64,9L,4,12523
17,0,547,10,7,64,7L,4,9044.53
21,0,174,10,7,16,Other,2,5576.69
40,0,128,13,5,32,Other,3,10678
50,0,157,10,7,32,Other,3,9263.8
62,0,827,0,1,64,9L,4,12523
69,0,640,8,2,64,7L,4,9031.4
85,0,970,8,2,64,9L,4,10144.1
94,0,615,13,5,32,honor,3,8456.8
125,0,318,11,4,32,honor,4,9514.86


#### Output

In [145]:
h2o.download_csv(dft_model0, "dft_model0.csv")

'/content/dft_model0.csv'

In [0]:
h2o_out = pd.read_csv('/content/dft_model0.csv')
h2o_out.set_index('C1', inplace=True)
#h2o_out.head()

In [147]:
model0_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1
15,0,361,0,1,64.0,9L,4.0
17,0,547,10,7,64.0,7L,4.0
21,0,174,10,7,16.0,Other,2.0
40,0,128,13,5,32.0,Other,3.0
50,0,157,10,7,32.0,Other,3.0


In [0]:
model0_test['Price'] = h2o_out['predict']

In [149]:
model0_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,Price
15,0,361,0,1,64.0,9L,4.0,12523.043945
17,0,547,10,7,64.0,7L,4.0,9044.527344
21,0,174,10,7,16.0,Other,2.0,5576.688965
40,0,128,13,5,32.0,Other,3.0,10677.984375
50,0,157,10,7,32.0,Other,3.0,9263.803711


#### Model Selection

In [0]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
model_ids

In [0]:
h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])

In [0]:
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])
out.convert_H2OXGBoostParams_2_XGBoostParams()

##Model1

### Data Preprocessing

In [0]:
IP = ['iphone x','iphonex','iphone 8','iphone 8 plus','iphone 7','iphone 7 plus',
      'iphone 6','iphone 6s','phone 7','phone 6','phone 8','7','6','8','7plus','6s','xs max','xs','5s',
      'iphone xs','iphonexs','iphone8','8plus','iphone7','iphone7plus',
      'iphone6','iphone6s','phone7','phone6','space grey','iphone','5','max','x','se','ipad','apple','xr','rose gold','watch','air pods','iphones','series','4s','4','iphones']
pat = '|'.join(r"\b{}\b".format(x) for x in IP)
model1['Model1'] = model1['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1['Model2'] = model1['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1.Model1.fillna(model1.Model2, inplace=True)
del model1['Model2']

model1["Model1"].replace({"iphone x": "X", "iphonex": "X", "x": "X", "xr": "X", "xs": "X", "xs max":"X", "iphonexs": "X", "iphone xs": "X","max": "X",
                          "iphone 8": "E", "iphone8": "E", "iphone 8 plus": "E", "phone 8": "E", "8": "E","8plus": "E",
                          "iphone 7": "E", "iphone7": "E", "iphone 7 plus": "E", "phone 7": "E", "7": "E", "7plus": "E","phone7": "E",
                          "iphone 6": "Si", "iphone6": "Si", "iphone 6s": "Si", "phone 6": "Si", "6": "Si", "6s": "Si","iphone6s": "Si",
                          "iphone 5": "F", "iphone5": "F", "5s": "F", "5": "F", "4": "F", "4s": "F","se" : "F","space grey":"F",
                          "iphone": "Si","ipad": "F","rose gold": "Si","series": "F","air pods": "F","apple": "F","iphones": "Si","watch":"F"
                          }, inplace=True)
model1['Model1'].fillna("Others" , inplace=True)
indices= model1['Price'] <= 25000
model1.loc[indices,'Storage'] = model1.loc[indices,'Storage'].fillna(32)
indices=  model1['Price'] <= 60000
model1.loc[indices,'Storage'] = model1.loc[indices,'Storage'].fillna(64)
model1['Storage'].fillna(256 , inplace=True)

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model1['RAM1'] = model1['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1['RAM2'] = model1['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1.RAM1.fillna(model1.RAM2, inplace=True)
del model1['RAM2']
model1["RAM1"].replace({"2gb": 2, "2 gb": 2,"3gb": 3,"3 gb": 3, "4gb": 4, "4 gb": 4, "6 gb": 6, "6gb": 6}, inplace=True)  
model1['RAM1'].fillna(3 , inplace=True)

In [0]:
M = ['iphone x','iphonex','iphone 8','iphone 8 plus','iphone 7','iphone 7 plus',
      'iphone 6','iphone 6s','phone 7','phone 6','phone 8','7','6','8','7plus','6s','xs max','xs','5s',
      'iphone xs','iphonexs','iphone8','8plus','iphone7','iphone7plus',
      'iphone6','iphone6s','phone7','phone6','space grey','iphone','5','max','x','se','ipad','apple','xr','rose gold','watch','air pods','iphones','series','4s','4','iphones']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model1_test['Model1'] = model1_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1_test['model1'] = model1_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1_test.Model1.fillna(model1_test.model1, inplace=True)
del model1_test['model1']
model1_test["Model1"].replace({"iphone x": "X", "iphonex": "X", "x": "X", "xr": "X", "xs": "X", "xs max":"X", "iphonexs": "X", "iphone xs": "X","max": "X",
                          "iphone 8": "E", "iphone8": "E", "iphone 8 plus": "E", "phone 8": "E", "8": "E","8plus": "E",
                          "iphone 7": "E", "iphone7": "E", "iphone 7 plus": "E", "phone 7": "E", "7": "E", "7plus": "E","phone7": "E",
                          "iphone 6": "Si", "iphone6": "Si", "iphone 6s": "Si", "phone 6": "Si", "6": "Si", "6s": "Si","iphone6s": "Si",
                          "iphone 5": "F", "iphone5": "F", "5s": "F", "5": "F", "4": "F", "4s": "F","se" : "F","space grey":"F",
                          "iphone": "Si","ipad": "F","rose gold": "Si","series": "F","air pods": "F","apple": "F","iphones": "Si","watch":"F"
                          }, inplace=True)
model1_test['Model1'].fillna("Others" , inplace=True)

model1_test['Storage'].fillna(32,inplace=True)   

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model1_test['RAM1'] = model1_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1_test['RAM2'] = model1_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model1_test.RAM1.fillna(model1_test.RAM2, inplace=True)
del model1_test['RAM2']
model1_test["RAM1"].replace({"2gb": 2,"2 gb": 2,"3gb": 3, "3 gb": 3,"4gb": 4, "4 gb": 4,"6 gb": 6, "6gb": 6}, inplace=True)  
model1_test['RAM1'].fillna(3 , inplace=True)

In [0]:
del model1['Additional_Description']
del model1['Model_Info']
del model1_test['Additional_Description']
del model1_test['Model_Info']

model1.to_csv("model1.csv",index=True)
model1_test.to_csv("model1_test.csv",index=True)

### Modelling

In [87]:
from h2o.automl import H2OAutoML
df_model1 = h2o.import_file('/content/model1.csv')
dft_model1 = h2o.import_file('/content/model1_test.csv')

df_model1['Brand'] = df_model1['Brand'].asfactor()
df_model1['Locality'] = df_model1['Locality'].asfactor()
df_model1['State'] = df_model1['State'].asfactor()
dft_model1['Brand'] = dft_model1['Brand'].asfactor()
dft_model1['Locality'] = dft_model1['Locality'].asfactor()
dft_model1['State'] = dft_model1['State'].asfactor()
ignored_col_list = "C1"

print(df_model1.types)
print(dft_model1.types)
df_train, df_val, df_test = df_model1.split_frame(ratios=[0.9,0.04])
y = "Price"
x = df_model1.columns
x.remove(y)
x.remove(ignored_col_list)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Price': 'int', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}


In [92]:
aml = H2OAutoML(nfolds=15,
                balance_classes=False,
                max_runtime_secs=240,
                max_models=0,
                exclude_algos=['DeepLearning','GLM'],
                verbosity = "info",
                sort_metric='RMSLE',
                stopping_metric='RMSLE')
aml.train(x=x,y=y,training_frame=df_train,validation_frame=df_val)
aml.leaderboard

AutoML progress: |
20:31:58.542: Project: AutoML_20200607_203158542
20:31:58.542: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:31:58.542: Setting stopping tolerance adaptively based on the training frame: 0.02297181937696983
20:31:58.542: Build control seed: -1 (random)
20:31:58.542: training frame: Frame key: automl_training_py_3424_sid_ab9d    cols: 9    rows: 1895  chunks: 8    size: 106273  checksum: -2477688304978400
20:31:58.543: validation frame: Frame key: py_3425_sid_ab9d    cols: 9    rows: 96  chunks: 8    size: 81728  checksum: -3416609799296862
20:31:58.543: leaderboard frame: NULL
20:31:58.543: blending frame: NULL
20:31:58.543: response column: Price
20:31:58.543: fold column: null
20:31:58.543: weights column: null
20:31:58.543: Loading execution steps: [{XG

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
XGBoost_grid__1_AutoML_20200607_203158_model_15,0.518648,2.05358e+08,14330.3,2.05358e+08,8640
XGBoost_grid__1_AutoML_20200607_203158_model_12,0.519293,1.87482e+08,13692.4,1.87482e+08,8364.98
StackedEnsemble_AllModels_AutoML_20200607_203158,0.522138,1.83578e+08,13549.1,1.83578e+08,8350.71
XGBoost_3_AutoML_20200607_203158,0.522473,1.92242e+08,13865.1,1.92242e+08,8440.95
XRT_1_AutoML_20200607_203158,0.524386,1.95635e+08,13987,1.95635e+08,8444.36
XGBoost_grid__1_AutoML_20200607_203158_model_5,0.524969,1.9272e+08,13882.4,1.9272e+08,8528.97
XGBoost_grid__1_AutoML_20200607_203158_model_8,0.525052,1.9024e+08,13792.7,1.9024e+08,8500.61
StackedEnsemble_BestOfFamily_AutoML_20200607_203158,0.525325,1.86619e+08,13660.9,1.86619e+08,8419.6
XGBoost_grid__1_AutoML_20200607_203158_model_1,0.528873,1.93848e+08,13922.9,1.93848e+08,8518.86
XGBoost_grid__1_AutoML_20200607_203158_model_4,0.529404,1.92654e+08,13880,1.92654e+08,8565.22


In [93]:
pred = aml.leader.predict(df_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [94]:
pred.head()

predict
19359.5
49213
13609.1
49213
12446.1
18810.9
13934.5
31273
61719.8
10090.6


In [95]:
aml.leader.model_performance(df_test)


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 104637737.93851574
RMSE: 10229.258914433427
MAE: 7219.014228515625
RMSLE: 0.4351338209870794
Mean Residual Deviance: 104637737.93851574


In [96]:
dft_model1['predict'] = aml.leader.predict(dft_model1)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [97]:
dft_model1

C1,Brand,Locality,City,State,Storage,Model1,RAM1,predict
0,1,570,11,4,32,Others,3,11534.4
1,1,762,8,2,256,F,3,39705.8
2,1,60,13,5,32,Others,3,13628
3,1,640,15,5,64,X,3,44090
4,1,816,2,6,32,F,4,10188
5,1,552,13,5,128,E,3,24188
6,1,389,8,2,64,Si,3,19660.7
7,1,926,8,2,32,F,3,11978.9
8,1,850,2,6,32,Si,3,13864.3
9,1,404,2,6,32,F,3,10188


#### Output

In [98]:
h2o.download_csv(dft_model1, "dft_model1.csv")

'/content/dft_model1.csv'

In [99]:
h2o_out = pd.read_csv('/content/dft_model1.csv')
h2o_out.set_index('C1', inplace=True)
h2o_out.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,predict
C1,,,,,,,,
0,1,570,11,4,32,Others,3,11534.413086
1,1,762,8,2,256,F,3,39705.828125
2,1,60,13,5,32,Others,3,13628.038086
3,1,640,15,5,64,X,3,44089.964844
4,1,816,2,6,32,F,4,10187.971680


In [100]:
model1_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1
0,1,570,11,4,32.0,Others,3.0
1,1,762,8,2,256.0,F,3.0
2,1,60,13,5,32.0,Others,3.0
3,1,640,15,5,64.0,X,3.0
4,1,816,2,6,32.0,F,4.0


In [0]:
model1_test['Price'] = h2o_out['predict']

In [102]:
model1_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,Price
0,1,570,11,4,32.0,Others,3.0,11534.413086
1,1,762,8,2,256.0,F,3.0,39705.828125
2,1,60,13,5,32.0,Others,3.0,13628.038086
3,1,640,15,5,64.0,X,3.0,44089.964844
4,1,816,2,6,32.0,F,4.0,10187.971680


##Model2

In [0]:
indices= model2['Price'] <= 2200
model2.loc[indices,'Storage'] = model2.loc[indices,'Storage'].fillna(16)
indices= model2['Price'] <= 5200
model2.loc[indices,'Storage'] = model2.loc[indices,'Storage'].fillna(32)
model2['Storage'].fillna(64 , inplace=True)


M = ['k5','k 5','k3','k 3','k4','k 4','k8 plus','k8plus','zuk','vibe p1m', 'vibep1m','k6','k 6','k8','k 8','vibe','note','phab','lenovo','iphone']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model2['Model1'] = model2['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2['Model2'] = model2['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2.Model1.fillna(model2.Model2, inplace=True)
del model2['Model2']
model2["Model1"].replace({"k5": "K5", "k 5": "K5",
                          "k4": "K4", "k 4": "K4",
                          "k3": "K3", "k 3": "K3",
                          "k2": "K2", "k 2": "K2",
                          "k8": "K8", "k 8": "K8",
                          "k6": "K6", "k 6": "K6",
                          "k8 plus": "K8", "k8plus": "K8",
                          "vibe p1m": "VIBE", "vibep1m": "VIBE","vibe":"VIBE",
                          "iphone": "APPLE",
                          "zuk": "LENOVO","note": "LENOVO","phab": "LENOVO","lenovo": "LENOVO"}, inplace=True)
model2['Model1'].fillna("Others" , inplace=True)


M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model2['RAM1'] = model2['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2['RAM2'] = model2['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2.RAM1.fillna(model2.RAM2, inplace=True)
del model2['RAM2']
model2["RAM1"].replace({"2gb": 2,"2 gb": 2, "3gb": 3, "3 gb": 3, "4gb": 4, "4 gb": 4,"6 gb": 6, "6gb": 6}, inplace=True)  

indices= model2['Storage'] == 16
model2.loc[indices,'RAM1'] = model2.loc[indices,'RAM1'].fillna(2)
indices= model2['Storage'] == 32
model2.loc[indices,'RAM1'] = model2.loc[indices,'RAM1'].fillna(3)
indices= model2['Storage'] == 64
model2.loc[indices,'RAM1'] = model2.loc[indices,'RAM1'].fillna(4)
model2['RAM1'].fillna(6 , inplace=True)

In [0]:
M = ['k5','k 5','k3','k 3','k4','k 4','k8 plus','k8plus','zuk','vibe p1m', 'vibep1m','k6','k 6','k8','k 8','vibe','note','phab','lenovo','iphone']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model2_test['Model1'] = model2_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2_test['Model2'] = model2_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2_test.Model1.fillna(model2_test.Model2, inplace=True)
del model2_test['Model2']
model2_test["Model1"].replace({"k5": "K5", "k 5": "K5", "k4": "K4", "k 4": "K4","k3": "K3", "k 3": "K3", "k2": "K2", "k 2": "K2", "k8": "K8", "k 8": "K8", "k6": "K6", "k 6": "K6", "k8 plus": "K8", "k8plus": "K8", "vibe p1m": "VIBE", "vibep1m": "VIBE","vibe":"VIBE", "iphone": "APPLE", "zuk": "LENOVO","note": "LENOVO","phab": "LENOVO","lenovo": "LENOVO"}, inplace=True)
model2_test['Model1'].fillna("Others" , inplace=True)

model2_test['Storage'].fillna(32,inplace=True)   

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model2_test['RAM1'] = model2_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2_test['RAM2'] = model2_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model2_test.RAM1.fillna(model2_test.RAM2, inplace=True)
del model2_test['RAM2']
model2_test["RAM1"].replace({"2gb": 2,"2 gb": 2,"3gb": 3, "3 gb": 3,"4gb": 4, "4 gb": 4,"6 gb": 6, "6gb": 6}, inplace=True)  

indices= model2_test['Storage'] == 16
model2_test.loc[indices,'RAM1'] = model2_test.loc[indices,'RAM1'].fillna(2)
indices= model2_test['Storage'] == 32
model2_test.loc[indices,'RAM1'] = model2_test.loc[indices,'RAM1'].fillna(3)
indices= model2_test['Storage'] == 64
model2_test.loc[indices,'RAM1'] = model2_test.loc[indices,'RAM1'].fillna(4)
model2_test['RAM1'].fillna(6 , inplace=True)

In [0]:
del model2['Additional_Description']
del model2['Model_Info']
del model2_test['Additional_Description']
del model2_test['Model_Info']

model2.to_csv("model2.csv",index=True)
model2_test.to_csv("model2_test.csv",index=True)

In [106]:
from h2o.automl import H2OAutoML
df_model2 = h2o.import_file('/content/model2.csv')
dft_model2 = h2o.import_file('/content/model2_test.csv')

df_model2['Brand'] = df_model2['Brand'].asfactor()
df_model2['Locality'] = df_model2['Locality'].asfactor()
df_model2['State'] = df_model2['State'].asfactor()
dft_model2['Brand'] = dft_model2['Brand'].asfactor()
dft_model2['Locality'] = dft_model2['Locality'].asfactor()
dft_model2['State'] = dft_model2['State'].asfactor()
ignored_col_list = "C1"

print(df_model2.types)
print(dft_model2.types)

df_train, df_val, df_test = df_model2.split_frame(ratios=[.9,.05])
y = "Price"
x = df_model2.columns
x.remove(y)
x.remove(ignored_col_list)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Price': 'int', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}


In [108]:
aml = H2OAutoML(nfolds=15,
                balance_classes=False,
                max_runtime_secs=240,
                max_models=0,
                exclude_algos=['DeepLearning','GLM'],
                verbosity = "info",
                sort_metric='RMSLE',
                stopping_metric='RMSLE')
aml.train(x=x,y=y,training_frame=df_train,validation_frame=df_val)
aml.leaderboard

AutoML progress: |
20:42:26.519: Project: AutoML_20200607_204226518
20:42:26.519: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:42:26.519: Setting stopping tolerance adaptively based on the training frame: 0.05
20:42:26.519: Build control seed: -1 (random)
20:42:26.519: training frame: Frame key: automl_training_py_4231_sid_ab9d    cols: 9    rows: 93  chunks: 1    size: 10854  checksum: -269037482319336
20:42:26.519: validation frame: Frame key: py_4232_sid_ab9d    cols: 9    rows: 3  chunks: 1    size: 9938  checksum: -191534906202178
20:42:26.519: leaderboard frame: NULL
20:42:26.519: blending frame: NULL
20:42:26.519: response column: Price
20:42:26.519: fold column: null
20:42:26.519: weights column: null
20:42:26.519: Loading execution steps: [{XGBoost : defaults}, {GL

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
XGBoost_grid__1_AutoML_20200607_204226_model_2,0.360366,3.03448e+06,1741.98,3.03448e+06,1156.31
GBM_grid__1_AutoML_20200607_204226_model_7,0.364476,2.8512e+06,1688.55,2.8512e+06,1122.36
GBM_3_AutoML_20200607_204226,0.36535,2.89668e+06,1701.96,2.89668e+06,1150.04
XGBoost_grid__1_AutoML_20200607_204226_model_1,0.365653,2.98533e+06,1727.81,2.98533e+06,1230.4
GBM_2_AutoML_20200607_204226,0.366189,2.94936e+06,1717.37,2.94936e+06,1136.08
GBM_grid__1_AutoML_20200607_204226_model_5,0.366444,2.95828e+06,1719.97,2.95828e+06,1125.91
XGBoost_grid__1_AutoML_20200607_204226_model_4,0.366649,3.17347e+06,1781.42,3.17347e+06,1147.72
GBM_1_AutoML_20200607_204226,0.367606,3.00888e+06,1734.61,3.00888e+06,1130.01
DRF_1_AutoML_20200607_204226,0.367626,3.09942e+06,1760.52,3.09942e+06,1178.64
XGBoost_2_AutoML_20200607_204226,0.370674,2.9384e+06,1714.18,2.9384e+06,1189.04


In [109]:
pred = aml.leader.predict(df_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [110]:
pred.head()

predict
4836.49
3286.17
4059.24
5664.41
5511.48
4187.28
3594.17
4328.39
5177.05


In [111]:
aml.leader.model_performance(df_test)


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 911740.8147728046
RMSE: 954.8512003306089
MAE: 739.7121310763889
RMSLE: 0.22601690672171643
Mean Residual Deviance: 911740.8147728046


In [112]:
dft_model2['predict'] = aml.leader.predict(dft_model2)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [113]:
dft_model2

C1,Brand,Locality,City,State,Storage,Model1,RAM1,predict
18,2,193,15,5,64,Others,4,7199.33
42,2,1129,11,4,32,VIBE,3,4320.36
73,2,769,2,6,32,K3,3,4736.18
108,2,1061,11,4,32,Others,3,4247.55
124,2,909,10,7,32,Others,4,4952.39
166,2,92,11,4,32,Others,4,5075.47
179,2,427,0,1,32,Others,3,4238.26
181,2,275,0,1,32,Others,3,4238.26
186,2,634,8,2,32,Others,3,4116.4
187,2,451,8,2,32,VIBE,3,4013.04


#### Output

In [114]:
h2o.download_csv(dft_model2, "dft_model2.csv")

'/content/dft_model2.csv'

In [115]:
h2o_out = pd.read_csv('/content/dft_model2.csv')
h2o_out.set_index('C1', inplace=True)
h2o_out.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,predict
C1,,,,,,,,
18,2,193,15,5,64,Others,4,7199.330078
42,2,1129,11,4,32,VIBE,3,4320.359375
73,2,769,2,6,32,K3,3,4736.182617
108,2,1061,11,4,32,Others,3,4247.548340
124,2,909,10,7,32,Others,4,4952.388672


In [116]:
model2_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1
18,2,193,15,5,64.0,Others,4.0
42,2,1129,11,4,32.0,VIBE,3.0
73,2,769,2,6,32.0,K3,3.0
108,2,1061,11,4,32.0,Others,3.0
124,2,909,10,7,32.0,Others,4.0


In [0]:
model2_test['Price'] = h2o_out['predict']

In [118]:
model2_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,Price
18,2,193,15,5,64.0,Others,4.0,7199.330078
42,2,1129,11,4,32.0,VIBE,3.0,4320.359375
73,2,769,2,6,32.0,K3,3.0,4736.182617
108,2,1061,11,4,32.0,Others,3.0,4247.548340
124,2,909,10,7,32.0,Others,4.0,4952.388672


##Model3

In [0]:
indices= model3['Price'] <= 10000
model3.loc[indices,'Storage'] = model3.loc[indices,'Storage'].fillna(32)
indices=  model3['Price'] <= 20000
model3.loc[indices,'Storage'] = model3.loc[indices,'Storage'].fillna(64)
model3['Storage'].fillna(128 , inplace=True)


M = ['lg','vivo','iphone','nexus']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model3['Model1'] = model3['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3['Model2'] = model3['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3.Model1.fillna(model3.Model2, inplace=True)
del model3['Model2']
model3["Model1"].replace({"lg": "LG", "vivo": "VIVO", "iphone": "APPLE","nexus": "LG" }, inplace=True)
model3['Model1'].fillna("Others" , inplace=True)

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model3['RAM1'] = model3['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3['RAM2'] = model3['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3.RAM1.fillna(model3.RAM2, inplace=True)
del model3['RAM2']
model3["RAM1"].replace({"2gb": 2,"2 gb": 2,"3gb": 3, "3 gb": 3,"4gb": 4, "4 gb": 4,"6 gb": 6, "6gb": 6}, inplace=True)  

indices= model3['Storage'] == 16
model3.loc[indices,'RAM1'] = model3.loc[indices,'RAM1'].fillna(2)
indices= model3['Storage'] == 32
model3.loc[indices,'RAM1'] = model3.loc[indices,'RAM1'].fillna(3)
indices= model3['Storage'] == 64
model3.loc[indices,'RAM1'] = model3.loc[indices,'RAM1'].fillna(4)
model3['RAM1'].fillna(6 , inplace=True)

In [0]:
M = ['lg','vivo','iphone','nexus']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model3_test['Model1'] = model3_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3_test['Model2'] = model3_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3_test.Model1.fillna(model3_test.Model2, inplace=True)
del model3_test['Model2']
model3_test["Model1"].replace({"lg": "LG", "vivo": "VIVO", "iphone": "APPLE","nexus": "LG" }, inplace=True)
model3_test['Model1'].fillna("Others" , inplace=True)

model3_test['Storage'].fillna(32,inplace=True)   

M = ['2gb','2 gb','3gb','3 gb','4 gb','4gb','6gb','6 gb']
pat = '|'.join(r"\b{}\b".format(x) for x in M)
model3_test['RAM1'] = model3_test['Model_Info'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3_test['RAM2'] = model3_test['Additional_Description'].str.extract('('+ pat + ')', expand=False, flags=re.I)
model3_test.RAM1.fillna(model3_test.RAM2, inplace=True)
del model3_test['RAM2']
model3_test["RAM1"].replace({"2gb": 2,"2 gb": 2,"3gb": 3, "3 gb": 3,"4gb": 4, "4 gb": 4,"6 gb": 6, "6gb": 6}, inplace=True)  

indices= model3_test['Storage'] == 16
model3_test.loc[indices,'RAM1'] = model3_test.loc[indices,'RAM1'].fillna(2)
indices= model3_test['Storage'] == 32
model3_test.loc[indices,'RAM1'] = model3_test.loc[indices,'RAM1'].fillna(3)
indices= model3_test['Storage'] == 64
model3_test.loc[indices,'RAM1'] = model3_test.loc[indices,'RAM1'].fillna(4)
model3_test['RAM1'].fillna(6 , inplace=True)

In [0]:
del model3['Additional_Description']
del model3['Model_Info']
del model3_test['Additional_Description']
del model3_test['Model_Info']

model3.to_csv("model3.csv",index=True)
model3_test.to_csv("model3_test.csv",index=True)

In [122]:
from h2o.automl import H2OAutoML
df_model3 = h2o.import_file('/content/model3.csv')
dft_model3 = h2o.import_file('/content/model3_test.csv')

df_model3['Brand'] = df_model3['Brand'].asfactor()
df_model3['Locality'] = df_model3['Locality'].asfactor()
df_model3['State'] = df_model3['State'].asfactor()
dft_model3['Brand'] = dft_model3['Brand'].asfactor()
dft_model3['Locality'] = dft_model3['Locality'].asfactor()
dft_model3['State'] = dft_model3['State'].asfactor()
ignored_col_list = "C1"

print(df_model3.types)
print(dft_model3.types)

df_train, df_val, df_test = df_model3.split_frame(ratios=[.8,.15])
y = "Price"
x = df_model3.columns
x.remove(y)
x.remove(ignored_col_list)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Price': 'int', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}
{'C1': 'int', 'Brand': 'enum', 'Locality': 'enum', 'City': 'int', 'State': 'enum', 'Storage': 'int', 'Model1': 'enum', 'RAM1': 'int'}


In [123]:
aml = H2OAutoML(nfolds=15,
                balance_classes=False,
                max_runtime_secs=240,
                max_models=0,
                exclude_algos=['DeepLearning','GLM'],
                verbosity = "info",
                sort_metric='RMSLE',
                stopping_metric='RMSLE')
aml.train(x=x,y=y,training_frame=df_train,validation_frame=df_val)
aml.leaderboard

AutoML progress: |
20:50:37.416: Project: AutoML_20200607_205037416
20:50:37.417: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:50:37.417: Setting stopping tolerance adaptively based on the training frame: 0.05
20:50:37.417: Build control seed: -1 (random)
20:50:37.417: training frame: Frame key: automl_training_py_4367_sid_ab9d    cols: 9    rows: 37  chunks: 1    size: 6022  checksum: -1764757347637578
20:50:37.417: validation frame: Frame key: py_4368_sid_ab9d    cols: 9    rows: 4  chunks: 1    size: 5688  checksum: -489787895203034
20:50:37.417: leaderboard frame: NULL
20:50:37.417: blending frame: NULL
20:50:37.417: response column: Price
20:50:37.417: fold column: null
20:50:37.417: weights column: null
20:50:37.418: Loading execution steps: [{XGBoost : defaults}, {GL

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
GBM_grid__1_AutoML_20200607_205037_model_3,0.654345,7.03902e+07,8389.89,7.03902e+07,4829.66
GBM_4_AutoML_20200607_205037,0.662791,7.77587e+07,8818.09,7.77587e+07,4948.06
GBM_3_AutoML_20200607_205037,0.671554,7.85603e+07,8863.42,7.85603e+07,5043.89
GBM_grid__1_AutoML_20200607_205037_model_11,0.676069,7.52108e+07,8672.41,7.52108e+07,5156.28
XGBoost_2_AutoML_20200607_205037,0.682896,7.66445e+07,8754.68,7.66445e+07,5008.05
DRF_1_AutoML_20200607_205037,0.683349,9.1375e+07,9559.03,9.1375e+07,5192.94
GBM_grid__1_AutoML_20200607_205037_model_29,0.68955,7.11588e+07,8435.57,7.11588e+07,4873.36
XGBoost_grid__1_AutoML_20200607_205037_model_4,0.690012,7.4531e+07,8633.13,7.4531e+07,4965.39
GBM_grid__1_AutoML_20200607_205037_model_6,0.691842,8.79348e+07,9377.36,8.79348e+07,5222.54
XGBoost_grid__1_AutoML_20200607_205037_model_22,0.694648,7.58156e+07,8707.22,7.58156e+07,4377.44


In [124]:
pred = aml.leader.predict(df_test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [125]:
pred.head()

predict
4357.06
24242.5
17134.2
4357.06


In [126]:
aml.leader.model_performance(df_test)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 31605950.087146264
RMSE: 5621.91694061254
MAE: 4455.341451921979
RMSLE: 0.658742341252289
Mean Residual Deviance: 31605950.087146264


In [127]:
dft_model3['predict'] = aml.leader.predict(dft_model3)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [128]:
dft_model3

C1,Brand,Locality,City,State,Storage,Model1,RAM1,predict
30,3,216,13,5,128,LG,6,20501
49,3,170,10,7,32,Others,3,3772.05
59,3,529,11,4,32,Others,3,6437.8
60,3,346,8,2,32,LG,3,4622.17
140,3,640,15,5,256,LG,6,24242.5
153,3,681,15,5,32,LG,3,6721.68
162,3,551,13,5,128,LG,6,16403.9
194,3,547,10,7,64,Others,4,13454.3
373,3,839,1,3,32,Others,3,3533.22
418,3,50,0,1,64,LG,4,13366.2


#### Output

In [129]:
h2o.download_csv(dft_model3, "dft_model3.csv")

'/content/dft_model3.csv'

In [130]:
h2o_out = pd.read_csv('/content/dft_model3.csv')
h2o_out.set_index('C1', inplace=True)
h2o_out.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,predict
C1,,,,,,,,
30,3,216,13,5,128,LG,6,20500.953858
49,3,170,10,7,32,Others,3,3772.053653
59,3,529,11,4,32,Others,3,6437.796977
60,3,346,8,2,32,LG,3,4622.170664
140,3,640,15,5,256,LG,6,24242.477750


In [131]:
model3_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1
30,3,216,13,5,128.0,LG,6.0
49,3,170,10,7,32.0,Others,3.0
59,3,529,11,4,32.0,Others,3.0
60,3,346,8,2,32.0,LG,3.0
140,3,640,15,5,256.0,LG,6.0


In [0]:
model3_test['Price'] = h2o_out['predict']

In [133]:
model3_test.head()

,Brand,Locality,City,State,Storage,Model1,RAM1,Price
30,3,216,13,5,128.0,LG,6.0,20500.953858
49,3,170,10,7,32.0,Others,3.0,3772.053653
59,3,529,11,4,32.0,Others,3.0,6437.796977
60,3,346,8,2,32.0,LG,3.0,4622.170664
140,3,640,15,5,256.0,LG,6.0,24242.477750


## Test

In [0]:
temp = dft

In [0]:
del temp['Price_m0']
del temp['Price_m1']
del temp['Price_m2']
del temp['Price_m3']

In [0]:
temp['Price_m0'] = model0_test['Price']
temp['Price_m1'] = model1_test['Price']
temp['Price_m2'] = model2_test['Price']
temp['Price_m3'] = model3_test['Price']
temp['Price_m0'].fillna(temp['Price_m1'] , inplace=True)
temp['Price_m0'].fillna(temp['Price_m2'] , inplace=True)
temp['Price_m0'].fillna(temp['Price_m3'] , inplace=True)

In [152]:
temp.head(60)

,Brand,Model_Info,Additional_Description,Locality,City,State,Storage,Price_m0,Price_m1,Price_m2,Price_m3
0,1,name0 55s66s66s778xxsxsmax etc,good condition 11months old single scratch we...,570,11,4,NaN,11534.413086,11534.413086,NaN,NaN
1,1,slightly used excellent condition name0 5 sale,101008700 1010030600 1010034300 10100192200 1...,762,8,2,256.0,39705.828125,39705.828125,NaN,NaN
2,1,name0 sx ios12 top letast model bill call,1010017300 delivery,60,13,5,NaN,13628.038086,13628.038086,NaN,NaN
3,1,name87 name0 x 64gb going lowest 41900,phone 1010023400 64 gb excellent condition sale,640,15,5,64.0,44089.964844,44089.964844,NaN,NaN
4,1,name0 5s proper condition one handedly used,full kit available 10100248300 condition 4gb ...,816,2,6,32.0,10187.971680,10187.971680,NaN,NaN
5,1,name0 7 plus 128 gb name75 gold,101006600 galaxy advance hai ok ram 512 call,552,13,5,128.0,24187.962891,24187.962891,NaN,NaN
6,1,brand new rosegold name87 iphone name234 64gb...,office gurgaon karol bagh new 101008700 10100...,389,8,2,64.0,19660.714844,19660.714844,NaN,NaN
7,1,name0 se 32gb,101008700 iphone 4s new brand refurbished pac...,926,8,2,32.0,11978.862305,11978.862305,NaN,NaN
8,1,name0 6s name753,101006200 bill good battery backup good looking,850,2,6,NaN,13864.305664,13864.305664,NaN,NaN
9,1,apple phone 8 offer price,brand new 101006600 galaxy s10 plus box best ...,404,2,6,NaN,10187.971680,10187.971680,NaN,NaN


In [153]:
sub = pd.read_excel('/content/Used_Electronics_Participants_Data/Sample_Submission.xlsx')
sub.head()

,Price
0,21773.398095
1,21277.682833
2,64172.642738
3,24174.489232
4,16453.462895


In [154]:
sub['Price'] = temp['Price_m0']
sub.head()

,Price
0,11534.413086
1,39705.828125
2,13628.038086
3,44089.964844
4,10187.971680


In [155]:
sub.describe()

,Price
count,997.000000
mean,22210.044985
std,13584.303383
min,2579.346680
25%,13428.382812
50%,18875.496094
75%,26040.193359
max,65432.230469


In [0]:
sub.to_excel('out.xlsx')